In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import os
from torchvision import datasets, transforms, models
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [2]:
def get_mean_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    for images,_ in loader:
        images_count_in_batch = images.size(0)
#         print(images.shape)
        images =images.view(images_count_in_batch,images.size(1),-1)
        mean+=images.mean(2).sum(0)
        std+=images.std(2).sum(0)
        total_images_count+=images_count_in_batch
    mean /= total_images_count
    std /= total_images_count
    return mean,std

In [9]:
def train_transform(dataset):
    train_transforms = transforms.Compose([transforms.Resize((299,299)),transforms.ToTensor()])
    train_dataset=torchvision.datasets.ImageFolder(root=dataset,transform=train_transforms)
    train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=32,shuffle=True)
    mean,std=get_mean_std(train_loader)
    train_transforms = transforms.Compose([
        transforms.Resize((299,299)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))
    ])
    train_dataset=torchvision.datasets.ImageFolder(root=dataset,transform=train_transforms)
    train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=32,shuffle=True,drop_last=False,num_workers=0)
    return train_loader,train_dataset

In [10]:
train_dataset='../Datasets/dataset_20_classes/'
train_loader,training_dataset=train_transform(train_dataset)

In [11]:
model = models.inception_v3(pretrained=False)
for param in model.parameters():
    param.requires_grad = False

In [12]:
import numpy as np

In [13]:
features = []
model.eval()
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        features.append(outputs.cpu().numpy())

features = np.concatenate(features)
labels = np.array(training_dataset.targets)

In [ ]:
# Apply t-SNE to the feature representations
tsne = TSNE(n_components=3, perplexity=30.0, n_iter=1000, random_state=42)
embedding = tsne.fit_transform(features)

# Visualize the embedding using a scatter plot
plt.scatter(embedding[:, 0], embedding[:, 1], c=labels)
plt.show()